In [ ]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from scipy.stats import linregress

import plotly.io as pio
import plotly.express as px
pio.templates.default = "plotly_white"

In [ ]:
def add_master_data_mappings(df: pd.DataFrame) -> pd.DataFrame:  
    # = Пути к справочникам - откорректировать если в реальной системе будут лежать по другому адресу =
    client_mapping_file = "data/client_mapping.csv"
    freight_mapping_file = "data/freight_mapping.csv"
    station_mapping_file = "data/station_mapping.csv"

    # Клиент - холдинг
    client_mapping = pd.read_csv(
        client_mapping_file,
        sep=";",
        decimal=",",
        encoding="windows-1251",
    )
    df = pd.merge(df, client_mapping, how="left", on="client_sap_id")

    # Груз
    freight_mapping = pd.read_csv(
        freight_mapping_file, sep=";", decimal=",", encoding="windows-1251"
    )
    df = pd.merge(df, freight_mapping, how="left", on="freight_id")

    # Станции
    station_mapping = pd.read_csv(
        station_mapping_file,
        sep=";",
        decimal=",",
        encoding="windows-1251",
    )
    df = pd.merge(
        df,
        station_mapping.add_prefix("sender_"),
        how="left",
        on="sender_station_id",
    )
    df = pd.merge(
        df,
        station_mapping.add_prefix("recipient_"),
        how="left",
        on="recipient_station_id",
    )
    
    df["holding_name"] = df["holding_name"].fillna(-1)
    return df

In [ ]:
df = pd.read_csv("data/fact_train_test.csv", sep=";", decimal=",", encoding="windows-1251")
df = add_master_data_mappings(df)

df["month"] = df["period"].str[5:7].astype(int)
df["year"] = df["period"].str[0:4].astype(int)

# Берем данные с 2021, для удобства обрежем сразу все данные
# df = df[ df["year"] >= 2021 ]

In [ ]:
# Признаки, по которым группируем при вычислении метрики
group_vars = [
    "period",
    "rps",
    "holding_name",
    "sender_department_name",
    "recipient_department_name",
]

grouped = df.groupby(group_vars, as_index=False).agg({"real_wagon_count": "sum", "real_weight": "sum"})
# grouped["real_density"] = grouped["real_weight"] / grouped["real_wagon_count"]

In [ ]:
# means = []
# stds = []
# r2s = []
# slopes = []

# for i in range(25):
    
#     row = grouped.sample(1).T.squeeze().to_dict()

#     a = grouped[(
#         (grouped["rps"] == row["rps"]) &
#         (grouped["holding_name"] == row["holding_name"]) &
#         (grouped["sender_department_name"] == row["sender_department_name"]) &
#         (grouped["recipient_department_name"] == row["recipient_department_name"])
#     )].sort_values("period")
    
#     if len(a) < 10:
#         print('---')
    
#     a["pred"] = a.tail(10)["real_wagon_count"].mean()
    
#     print(row)
#     fig = px.line(a, x="period", y=["real_wagon_count", "pred"])
#     fig.update_layout(showlegend=False)
#     fig.show()
    
#     means.append( a["real_wagon_count"].mean() )
#     stds.append( a["real_wagon_count"].std() )
#     r2s.append( trend.rvalue )
#     slopes.append( trend.slope )

# Median2021 submit

In [ ]:
print(group_vars)

In [ ]:
# За основу возьмем их сабмит
df_medians = pd.read_csv("data/forecast_example.csv", sep=";", decimal=",", encoding="windows-1251")
original_columns = df_medians.columns

df_medians = add_master_data_mappings(df_medians)
# Оставим только уникальные строчки по хэшам
df_medians = df_medians.drop_duplicates(subset=group_vars)
df_medians = df_medians.drop(columns=["forecast_weight", "forecast_wagon_count"])

In [ ]:
df_medians

In [ ]:
list_count = []
list_weight = []

for i, (period, rps, holding_name, sender_department_name, recipient_department_name) in enumerate(df_medians[group_vars].values):
    if i % 100 == 0:
        print(i, end="\r")
        
    a = grouped[(
        (grouped["rps"] == rps) &
        (grouped["holding_name"] == holding_name) &
        (grouped["sender_department_name"] == sender_department_name) &
        (grouped["recipient_department_name"] == recipient_department_name)
    )].sort_values("period")
    
    if len(a) >= 10:
        list_count.append( a.tail(10)["real_wagon_count"].mean() )
        list_weight.append( a.tail(10)["real_weight"].mean() )
    else:
        list_count.append( a["real_wagon_count"].mean() )
        list_weight.append( a["real_weight"].mean() )
    
df_medians["forecast_wagon_count"] = list_count
df_medians["forecast_weight"] = list_weight

In [ ]:
df_medians[original_columns].to_csv("submits/10_last_mean.csv", index=False, sep=";", decimal=",", encoding="windows-1251")